In [28]:
import keras
import numpy as np
import csv
import nltk
nltk.download('punkt')
import pandas as pd
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence


data=[]
testa=[]

#Upload the training data
with open('olid-training-v1.0.tsv') as tsvfile:
  reader = csv.reader(tsvfile, delimiter='\t')
  for line in reader:
    ids = line[0]
    tweet = line[1]
    subtask_a = line[2]
    subtask_b = line[3]
    subtask_c = line[4]
    data.append((ids,tweet,subtask_a,subtask_b,subtask_c))

df = pd.DataFrame(data[1:],columns=['id','tweet','subtask_a','subtask_b','subtask_c'])

#upload the test data
with open('testset-levela.tsv') as tsvfilea:
  readera = csv.reader(tsvfilea, delimiter='\t')
  for line in readera:
    ids = line[0]
    tweet = line[1]
    testa.append((ids,tweet))

dftesta = pd.DataFrame(testa[1:],columns=['id','tweet'])

#store the training labels to a list
all_labels=[]

all_labels=pd.concat([df['subtask_a'],df['subtask_b'],df['subtask_c']], axis=1)
all_labels=all_labels.values.tolist()


#upload test labels and store them to a list
testlabels=[]

dflaba = pd.read_csv('labels-levela.csv', names=['ids', 'label'])
dflabb = pd.read_csv('labels-levelb.csv', names=['ids', 'label'])
dflabc = pd.read_csv('labels-levelc.csv', names=['ids', 'label'])

test_labels=pd.concat([dflaba['label'],dflabb['label'],dflabc['label']] ,axis=1)
test_labels['label'] = test_labels['label'].astype(str)
test_labels=test_labels.values.tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
#Preprocessing function


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import string
import re
nltk.download('stopwords')
def preprocessing(text):

    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    #print(tokens)


   #remove punctuation
    #pattern = '[0-9]'
    # tokens = [re.sub(r"[^a-zA-Z0-9]", '', text) for t in text]
    #tokens = [re.sub(r'\s([?.!"](?:\s|$))', r'\1',t) for t in text]
   # tokens = [re.sub(pattern, '', tokens) for t in tokens]

 
    # lowercasing
    tokens = [t.lower() for t in tokens]

    # stopword removal- benefits are it removes rare words
    stop = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop]
    #print(tokens)

  
    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#Preprocess the training data and store them to a list
for index, line in df.iterrows():
  #print(preprocessing(line.tweet))
  df['tweet'][index] = preprocessing(str(line.tweet))

normalized=[]

for index, line in df.iterrows():
  normalized.append(df['tweet'][index])

#Preprocess the test data and store them to a list
for index, line in dftesta.iterrows():
  #print(preprocessing(line.tweet))
  dftesta['tweet'][index] = preprocessing(str(line.tweet))

normalized_testa=[]

for index, line in dftesta.iterrows():
  normalized_testa.append(dftesta['tweet'][index])



In [0]:
#word2idx,idx2word,sents_as_ids for training

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(normalized)

word2idx = tokenizer.word_index
word2idx["<PAD>"] = 0
idx2word = {v:k for k, v in word2idx.items()}

sents_as_ids=[]
for line in range(len(normalized)):
    wids=[]
    for a in normalized[line]:
      wids.append(word2idx[a])
    sents_as_ids.append(wids)

#word2idx,idx2word,sents_as_ids for testing
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(normalized_testa)

word2idxa = tokenizer.word_index
word2idxa["<PAD>"] = 0
idx2worda = {v:k for k, v in word2idxa.items()}

sents_as_idsa=[]
for line in range(len(normalized_testa)):
    widsa=[]
    for a in normalized_testa[line]:
      widsa.append(word2idxa[a])
    sents_as_idsa.append(widsa)

vocab_size = len(word2idx)+1
embed_size = 100

In [32]:
print('Number of unique words:', len(idx2word))
print('\nVocabulary Sample:', list(idx2word.items())[:10])
print('\nSample word2idx: ', list(word2idx.items())[:10])
print('\nAbove sentence as a list of ids:' , sents_as_ids[:3])


Number of unique words: 18974

Vocabulary Sample: [(1, 'user'), (2, 'url'), (3, 'liberals'), (4, 'gun'), (5, 'control'), (6, 'antifa'), (7, 'like'), (8, 'maga'), (9, 'conservatives'), (10, 'people')]

Sample word2idx:  [('user', 1), ('url', 2), ('liberals', 3), ('gun', 4), ('control', 5), ('antifa', 6), ('like', 7), ('maga', 8), ('conservatives', 9), ('people', 10)]

Above sentence as a list of ids: [[1, 256, 2498, 184, 45], [1, 1, 22, 205, 834, 1, 8, 772, 2], [3486, 6220, 1732, 2285, 1643, 4913, 1305, 1733, 48, 8875, 189, 3487, 3488, 4914, 2, 3486, 8, 272, 721, 395]]


In [0]:
#TASKA binary classification - encoding the labels

#encode the label
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()

#encode the labels for TaskA
binarylabels = Encoder.fit_transform(df['subtask_a'])
testbinarylabels = Encoder.fit_transform(dflaba['label'])


In [0]:
#TASKA,B,C MULTI classification - encoding the labels

#word2idx,idx2word,sents_as_ids for training labels, we need to translate the labels to numbers for the model

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(all_labels)

word2idxm = tokenizer.word_index
word2idxm["<PAD>"] = 0
idx2wordm = {v:k for k, v in word2idxm.items()}

sents_as_idsm=[]
for line in range(len(all_labels)):
    widsm=[]
    for a in all_labels[line]:
        for w in text.text_to_word_sequence(a):
          widsm.append(word2idxm[w])
    sents_as_idsm.append(widsm)

#word2idx,idx2word,sents_as_ids for testing labels, we need to translate the labels to numbers for the model
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(test_labels)

word2idxt = tokenizer.word_index
word2idxt["<PAD>"] = 0
idx2wordt = {v:k for k, v in word2idxt.items()}

sents_as_idst=[]
for line in range(len(test_labels)):
    widst=[]
    for a in test_labels[line]:
        for w in text.text_to_word_sequence(a):
          widst.append(word2idxt[w])
    sents_as_idst.append(widst)

multilabels=np.array(sents_as_idsm)
testmultilabels=np.array(sents_as_idst)

In [0]:
#Preparing input data

from keras.preprocessing.sequence import pad_sequences
MAXIMUM_LENGTH = 256


preprocessed_train_data = keras.preprocessing.sequence.pad_sequences(sents_as_ids,
                                                        value=word2idx["<PAD>"],
                                                        padding='post',
                                                        maxlen=MAXIMUM_LENGTH)


preprocessed_test_data = keras.preprocessing.sequence.pad_sequences(sents_as_idsa,
                                                        value=word2idxa["<PAD>"],
                                                        padding='post',
                                                        maxlen=MAXIMUM_LENGTH)

In [36]:
print(preprocessed_test_data[1])

[ 608 1728    3 1729  336  461   23   16   97    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [0]:
#LSTM model for binary classification

from keras import models
from keras import layers
from keras.layers import Embedding,LSTM

EMBED_SIZE = 100

MAXIMUM_LENGTH = 256

model = models.Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=EMBED_SIZE,input_length=MAXIMUM_LENGTH))
model.add(LSTM(units=100, activation='tanh'))   #change activation function to tanh
model.add(layers.Dense(1, activation='sigmoid'))



model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
 print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 256, 100)          1897500   
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,978,001
Trainable params: 1,978,001
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
#LSTM model for multi classification

model2 = models.Sequential()
model2.add(Embedding(input_dim=vocab_size,output_dim=EMBED_SIZE,input_length=MAXIMUM_LENGTH))
model2.add(LSTM(units=100, activation='tanh'))   #change activation function to tanh
model2.add(layers.Dense(3, activation='sigmoid'))



model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [40]:
 print(model2.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 256, 100)          1897500   
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 303       
Total params: 1,978,203
Trainable params: 1,978,203
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
#When training, we want to check the accuracy of the model on data it hasn't seen before. So we create a validation set

X_val = np.array(preprocessed_train_data[:2000])
partial_X_train = np.array(preprocessed_train_data[2000:])

y_val = np.array(binarylabels[:2000])
partial_y_train = np.array(binarylabels[2000:])

y_val_m = np.array(multilabels[:2000])
partial_y_train_m = np.array(multilabels[2000:])

print(X_val)

[[    1   256  2498 ...     0     0     0]
 [    1     1    22 ...     0     0     0]
 [ 3486  6220  1732 ...     0     0     0]
 ...
 [    1     1 10409 ...     0     0     0]
 [    1   831   122 ...     0     0     0]
 [    1  1863     0 ...     0     0     0]]


In [42]:
#Then we start to train the model for 3 epochs in mini-batches of 100 samples and monitor the model's loss and accuracy on the validation set.


history = model.fit(partial_X_train,
                    partial_y_train,
                    epochs=3,
                    batch_size=100,
                    validation_data=(X_val, y_val),
                    verbose=1)


history = model2.fit(partial_X_train,
                    partial_y_train_m,
                    epochs=3,
                    batch_size=100,
                    validation_data=(X_val, y_val_m),
                    verbose=1)

Train on 11240 samples, validate on 2000 samples
Epoch 1/3
11240/11240 [==============================] - 72s 6ms/step - loss: 0.6395 - acc: 0.6653 - val_loss: 0.6395 - val_acc: 0.6630
Epoch 2/3
11240/11240 [==============================] - 71s 6ms/step - loss: 0.6357 - acc: 0.6685 - val_loss: 0.6392 - val_acc: 0.6630
Epoch 3/3
11240/11240 [==============================] - 71s 6ms/step - loss: 0.6360 - acc: 0.6685 - val_loss: 0.6391 - val_acc: 0.6630
Train on 11240 samples, validate on 2000 samples
Epoch 1/3
11240/11240 [==============================] - 72s 6ms/step - loss: 7.4424 - acc: 0.4752 - val_loss: 7.5116 - val_acc: 0.3035
Epoch 2/3
11240/11240 [==============================] - 71s 6ms/step - loss: 7.4417 - acc: 0.4326 - val_loss: 7.5113 - val_acc: 0.6630
Epoch 3/3
11240/11240 [==============================] - 71s 6ms/step - loss: 7.4418 - acc: 0.4884 - val_loss: 7.5114 - val_acc: 0.6630


In [46]:
results = model.evaluate(preprocessed_test_data, testbinarylabels)

print(results)

860/860 [==============================] - 3s 3ms/step
[0.5985215977180836, 0.7209302331126013]


In [44]:
results2 = model2.evaluate(preprocessed_test_data, testmultilabels)

print(results2)

860/860 [==============================] - 2s 2ms/step
[7.025197723299958, 0.7209302325581395]


In [0]:
#optimise
from keras.layers import Dense, Activation, Dropout

model3 = models.Sequential()
model3.add(Embedding(input_dim=vocab_size,output_dim=EMBED_SIZE,input_length=MAXIMUM_LENGTH))
#model3.add(Activation('relu'))
model3.add(LSTM(units=100, activation='tanh'))   #change activation function to tanh
model3.add(Dropout(0.5))
#model3.add(layers.Dense(3, activation='softmax'))
model3.add(layers.Dense(3, activation='sigmoid'))

model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [71]:
history = model3.fit(partial_X_train,
                    partial_y_train_m,
                    epochs=3,
                    batch_size=100,
                    validation_data=(X_val, y_val_m),
                    verbose=1)

Train on 11240 samples, validate on 2000 samples
Epoch 1/3
11240/11240 [==============================] - 77s 7ms/step - loss: 7.4425 - acc: 0.5109 - val_loss: 7.5110 - val_acc: 0.3035
Epoch 2/3
11240/11240 [==============================] - 73s 7ms/step - loss: 7.4421 - acc: 0.4714 - val_loss: 7.5124 - val_acc: 0.6630
Epoch 3/3
11240/11240 [==============================] - 73s 6ms/step - loss: 7.4421 - acc: 0.4601 - val_loss: 7.5133 - val_acc: 0.6630


In [72]:
results = model3.evaluate(preprocessed_test_data, testmultilabels)

print(results)

860/860 [==============================] - 2s 2ms/step
[7.024758487523989, 0.7209302325581395]


In [64]:
# from keras.layers import Dense, Conv2D, Flatten

# model3 = models.Sequential()
# model3.add(Embedding(input_dim=vocab_size,output_dim=EMBED_SIZE,input_length=MAXIMUM_LENGTH))
# model.add(Conv2D(64, kernel_size=3, activation='relu'))
# model.add(Conv2D(32, kernel_size=3, activation='relu'))
# model.add(Flatten())
# model.add(Dense(10, activation='softmax'))

ValueError: ignored